<a href="https://colab.research.google.com/github/ani-sh2301/DL-projects/blob/main/Sign_Language_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'include-50:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2991055%2F5148069%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240529%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240529T055615Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2a5805005409c1025e0fa638f33017dd5792d0dbe6c257821b2bb533a9a7bc8cbbe897d6638c017b71cbbf1b8f657ae5a0fc9698d3f62a6cbfe15ecfbd1ae0c807a3f5ebabaa4d85f22390a595742cb49a1a7846ececfaedeb697ed1937ee7857b6801b6f736af9895705f23de90da2597e910f7378bf7a9b7e0484835e2c4ac44b1bfec6d5bb01e921ab2048a8ce4e85efabf4665561e0b3562cae2d8c5e6922acdf097062b739959c9eebd6e59468b21180c1b91f0bff1334871581b5fd442d8ff3da38038abe0cdd0c25623b01ec1df020ca03ed17777e301f10c192af402ccdf2eab47a9e5df42f8bb7a47536a24dc3de0ca9664d4a93c3d629cc0214c6b'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install --upgrade pip
!pip install mediapipe
!pip install tensorboard
!pip install opencv-python

In [ ]:
#!apt-get update && apt-get install -y python3-opencv
#!pip install opencv-python

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
import mediapipe as mp
import pandas as pd
import random
from tqdm import tqdm
from zipfile import ZipFile
from progressbar import ProgressBar
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow.keras.utils import to_categorical
pb=ProgressBar()



In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from cv2 import resize
from keras.models import Model
from keras.applications.vgg16 import preprocess_input

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df=pd.DataFrame(columns=["path","label"])
df

In [ ]:
path='/kaggle/input/include-50'
for files in os.listdir(path):
    f=os.path.join(path,files)
    for labels in os.listdir(f):
        c=os.path.join(f,labels)
        for clips in os.listdir(c):
            if clips.lower()=="extra":
                ac=os.path.join(c,clips)
                for x in os.listdir(ac):
                    row={"path": os.path.join(ac,x), 'label':(labels.split(".")[1]).strip()}
                    df.loc[len(df.index)] = row
            else:
                row={"path": os.path.join(c,clips), 'label':(labels.split(".")[1]).strip()}
                df.loc[len(df.index)] = row



In [ ]:
seed=random.seed(1024)
print(random.randint(7,12))
shuffle_df=df.sample(frac = 1,random_state=random.randint(7,12), ignore_index=True)

In [ ]:
df['path'][0]

In [ ]:
shuffle_df.head()

In [ ]:
def drawLandmarks(image, result):
    #mp_drawings.draw_landmarks(image, result.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
     #                         mp_drawings.DrawingSpec(color=(80,110,10), thickness=1,circle_radius=1),
      #                        mp_drawings.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
       #                       )
    mp_drawings.draw_landmarks(image, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawings.DrawingSpec(color=(80,22,10), thickness=1,circle_radius=3),
                              mp_drawings.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=2)
                              )
    mp_drawings.draw_landmarks(image, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawings.DrawingSpec(color=(80,33,77), thickness=1,circle_radius=3),
                              mp_drawings.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=2)
                              )
    mp_drawings.draw_landmarks(image, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawings.DrawingSpec(color=(80,117,66), thickness=1,circle_radius=3),
                              mp_drawings.DrawingSpec(color=(80,66,239), thickness=1, circle_radius=2)
                              )

In [ ]:
def detectKeyPoints(image,model):
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    return image,results

In [ ]:
def fps30(cap):
    keypoints=np.array([[0]*258])
    #print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            #print(frame_idx)
            if len(keypoints)>=31: break

            ret, frame = cap.read()
            image, results=detectKeyPoints(frame[:,500:1500],holistic)
            keypoints=np.append(keypoints,[extract_keypoints(results)],axis=0)
    return keypoints[1:]

In [ ]:
def fps60(cap):
    keypoints=np.array([[0]*258])
    #print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            #print(frame_idx)
            if len(keypoints)>=31: break

            if frame_idx%2==0:
                ret, frame = cap.read()
                image, results=detectKeyPoints(frame[:,500:1500],holistic)
                keypoints=np.append(keypoints,[extract_keypoints(results)],axis=0)
    return keypoints[1:]

In [ ]:
def fps90(cap):
    keypoints=np.array([[0]*258])
    #print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            #print(frame_idx)
            if len(keypoints)>=31: break

            if frame_idx%3==0:
                ret, frame = cap.read()
                image, results=detectKeyPoints(frame[:,500:1500],holistic)
                keypoints=np.append(keypoints,[extract_keypoints(results)],axis=0)
    return keypoints[1:]

In [ ]:
def fps120(cap):
    keypoints=np.array([[0]*258])
    #print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            #print(frame_idx)
            if len(keypoints)>=31: break

            if frame_idx%4==0:
                ret, frame = cap.read()
                image, results=detectKeyPoints(frame[:,500:1500],holistic)
                keypoints=np.append(keypoints,[extract_keypoints(results)],axis=0)
    return keypoints[1:]

In [ ]:
def fps150(cap):
    keypoints=np.array([[0]*258])
    #print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            #print(frame_idx)
            if len(keypoints)>=31: break

            if frame_idx%5==0:
                ret, frame = cap.read()
                image, results=detectKeyPoints(frame[:,500:1500],holistic)
                keypoints=np.append(keypoints,[extract_keypoints(results)],axis=0)
    return keypoints[1:]


In [ ]:
def greaterThan90(cap):
    keypoints=np.array([[0]*258])
    #print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames=[]
    while True:
        ret, frame=cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    r=(len(frames)-90)//2
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_idx in range(r,(90+r)) :
            #print(frame_idx)
            if len(keypoints)>=31: break

            if frame_idx%2==0:
                #ret, frame = cap.read()
                image, results=detectKeyPoints(frames[frame_idx][:,500:1500],holistic)
                keypoints=np.append(keypoints,[extract_keypoints(results)],axis=0)
    return keypoints[1:]


In [ ]:
def extract_From_KeyFrames(frames):
    keypoints=np.array([[0]*258])
    #print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    with mp_holistic.Holistic(min_detection_confidence=0.35, min_tracking_confidence=0.5) as holistic:
        for frame in frames :
                #ret, frame = cap.read()
                image, results=detectKeyPoints(frame[:,500:1500],holistic)
                keypoints=np.append(keypoints,[extract_keypoints(results)],axis=0)
    return keypoints[1:]

In [ ]:
def preprocess(file_path,model, n_key_frames = 15):

    frames = []
    cap = cv2.VideoCapture(file_path)

    while True:
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break

    features = []
    for frame in frames:
        frame = resize(frame, (224, 224))
        frame = frame.reshape(1, 224, 224, 3)
        img= preprocess_input(frame)
        feat = model.predict(frame, verbose=0)
        features.append(feat)
    features = np.array(features)
    feats = features.reshape(-1, 4096)
    pac_man = PCA(n_components=None, random_state=69)

    dfeats = pac_man.fit_transform(feats)

    groups = {}
    kmeans = KMeans(n_clusters=n_key_frames, random_state = 22)
    kmeans.fit(dfeats)

    for index, cluster in zip(range(len(dfeats)), kmeans.labels_):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(index)
        else:
            groups[cluster].append(index)
    key_frames = []

    for key, values in groups.items():
        key_frames.append(groups[key][0])

    final_frames = []
    for key in key_frames:
        final_frames.append(frames[key])
    return final_frames

In [ ]:
def extract_keypoints(results):
    pose=np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face=np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    rh=np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh=np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose,lh,rh])

In [ ]:
df.head()

In [ ]:
label_map={label:num for num,label in enumerate(df['label'].unique())}
label_map

In [ ]:

mp_holistic= mp.solutions.holistic
mp_drawings= mp.solutions.drawing_utils
kf = VGG16()
kf = Model(inputs=kf.inputs, outputs=kf.layers[-2].output)

In [ ]:
'''
Code to extract and save keyframes as numpy array files


pb=ProgressBar()
keyframes, label=[],[]
count=0
for i, row in tqdm(shuffle_df.iterrows()):
    keyframes=(preprocess(row["path"], model=kf,n_key_frames=30))
    #label.append(label_map[row['label']])
    np.save(f"{count} {row['label']}",np.array(keyframes)/255)
    count+=1

'''

In [ ]:
#code to lad Key Frames suing Numpy Array Files

"""
keyframes=[]
label=[]
path='/kaggle/input/include-keyframes'
for file in tqdm(os.listdir(path)):
    if 'results' in file :
        continue
    else:
        file_path=os.path.join(path,file)
        keyframes.append(np.load(file_path))
        l=" ".join(file.split(" ")[1:]).strip()
        label.append(label_map["".join(l.split(".")[0].strip())])

"""

In [ ]:
"""
pb=ProgressBar()
features=[]
for frames in pb(keyframes):
    features.append(extract_From_KeyFrames(frames))


"""

In [ ]:
"""
X,y=[],[]
for i, row in tqdm(shuffle_df.iterrows()):
    keyframes=(preprocess(row["path"], model=kf,n_key_frames=30))
    X.append(extract_From_KeyFrames(keyframes))
    y.append(label_map[row['label']])
"""

In [ ]:
#y_cat=to_categorical(y).astype(int)

In [ ]:
#X_train, X_test, y_train, y_test=train_test_split(X,y_cat, test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
tf.random.set_seed(69)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
  ax1.plot(history.history['loss'], label='loss')
  ax1.plot(history.history['val_loss'], label='val_loss')
  ax1.set_xlabel('Epoch')
  ax1.set_ylabel('kullback_leibler_divergence')
  ax1.grid(True)

  ax2.plot(history.history['accuracy'], label='accuracy')
  ax2.plot(history.history['val_accuracy'], label='val_accuracy')
  ax2.set_xlabel('Epoch')
  ax2.set_ylabel('Accuracy')
  ax2.grid(True)

  plt.show()

In [ ]:
log_dir=os.path.join('Logs')
tb_callback=TensorBoard(log_dir=log_dir)

In [ ]:
def train_model(X_train, y_train, bs, dr, lr, e):
    model=Sequential()
    model.add(BatchNormalization(input_shape=(30,258)))
    model.add(LSTM(64, return_sequences=True, activation='swish'))
    model.add(Dropout(dr))
    model.add(LSTM(128, return_sequences=True, activation='swish'))
    model.add(Dropout(dr))
    model.add(LSTM(256, return_sequences=True, activation='swish'))
    model.add(Dropout(dr))
    model.add(LSTM(512, return_sequences=True, activation='swish'))
    model.add(Dropout(dr))
    model.add(BatchNormalization())
    model.add(LSTM(256, return_sequences=True, activation='swish'))
    model.add(Dropout(dr))
    model.add(LSTM(128, return_sequences=True, activation='swish'))
    model.add(BatchNormalization())
    model.add(LSTM(64, return_sequences=True, activation='swish'))
    model.add(Dropout(dr))
    model.add(LSTM(32, return_sequences=False, activation='swish'))
    model.add(BatchNormalization())
    model.add(Dense(64, activation='swish'))
    model.add(Dropout(dr))
    model.add(Dense(128, activation='swish'))
    model.add(Dropout(dr))
    model.add(Dense(128, activation='swish'))
    model.add(Dropout(dr))
    model.add(BatchNormalization())
    model.add(Dense(64,activation='swish'))
    model.add(Dropout(dr))
    model.add(Dense(40,activation='softmax'))


    model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=lr),
                                loss='kullback_leibler_divergence',
                                metrics=['accuracy'])
    history=model.fit(X_train,y_train, epochs=e, validation_split=0.2, batch_size=bs, verbose=0)

    return model, history

In [ ]:
#X_train=tf.constant(X_train, dtype='float64')
#Y_train=tf.constant(y_train, dtype='int32')
#X_test=tf.constant(X_test, dtype='float64')
#Y_test=tf.constant(y_test, dtype='int32')

In [ ]:
"""
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)
"""

In [ ]:
"""

np.save("X_train",X_train)
np.save("y_train",y_train)
np.save("X_test",X_test)
np.save("y_test",y_test)

"""

In [ ]:
X_train=np.load('/kaggle/input/train-data-keypoints/X_train.npy')
y_train=np.load('/kaggle/input/train-data-keypoints/y_train.npy')
X_test=np.load('/kaggle/input/train-data-keypoints/X_test.npy')
y_test=np.load('/kaggle/input/train-data-keypoints/y_test.npy')

In [ ]:
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
batch_size = [15, 20, 32]
epochs = [ 100, 150, 200, 300]
learn_rate = [0.001, 0.01, 0.1]
dropout= [0.0, 0.1, 0.2, 0.3]

In [ ]:

all_history=[]
least_val_loss = float('inf')
least_loss_model = None
for bs in batch_size:
    for e in epochs:
        for lr in learn_rate:
            for dr in dropout:
                print(f"LR {lr}, Epochs {e}, Batch Size {bs}, Dropout {dr}")
                model, history = train_model(X_train, y_train, bs, dr, lr, e)
                plot_history(history)
                val_loss = model.evaluate(X_test, y_test)[0]
                if val_loss < least_val_loss:
                  least_val_loss = val_loss
                  least_loss_model = model
                all_history.append(dict(batch_size= bs,
                                       epoch=e,
                                       learning_rate= lr,
                                       #momentum=m,
                                       history= history))
                #print("***************************************************************************")


In [ ]:
"""
Code to clip and normalize loss values

temp=np.array(history.history['val_loss'])
temp[np.isnan(temp)]=0
temp=np.clip(temp,0,20)
history.history['val_loss']=temp

"""

In [ ]:
model, history = train_model(X_train, y_train, 15, 0.3, 0.001, 150)

In [ ]:
plot_history(history)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
import pickle
pickle.dump(model,open("beta_model.sav","wb"))

In [ ]:
import torch
torch.save(model,"beta_model")

In [ ]:
#res=least_loss_model.predict(X_test)

In [ ]:
#np.argmax(res[0])

In [ ]:
#decode=list(label_map.keys())

In [ ]:
#print(decode[np.argmax(res[2])])
#print(decode[np.argmax(y_test[2])])

In [ ]:
#from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
#ypred=np.argmax(res, axis=1).tolist()
#ytrue=np.argmax(y_test, axis=1).tolist()

In [ ]:
#print(ypred, ytrue)

In [ ]:
#multilabel_confusion_matrix(ytrue, ypred)

In [ ]:
#accuracy_score(ytrue, ypred)

In [ ]:
#keyframes=np.load('/kaggle/input/include-keyframes-2/0 How are you.npy')

In [ ]:
#kf=(keyframes*255).astype(int)
#kf.shape
#detectKeyPoints(kf[:,500:1500],holistic)

In [ ]:
#kf.shape

In [ ]:
"""
code to plot keyframes along with Keypoints

fig_rows = 10
fig_cols = 2
n_subplots = fig_rows * fig_cols
plot_range = n_subplots
start_stop = 0

fig, axs = plt.subplots(fig_rows, fig_cols, figsize=[50, 100])

with mp_holistic.Holistic(min_detection_confidence=0.35, min_tracking_confidence=0.3) as holistic:
    for idx, img in enumerate(range(start_stop, plot_range)):
        image, results=detectKeyPoints(kf[idx][:,500:1500],holistic)
        drawLandmarks(image, results)
        axs.flat[idx].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axs.flat[idx].axis('off')
        axs.flat[idx].set_title(decode[label[0]])

plt.tight_layout()
plt.show()

"""